# pycrpto

In [24]:
import json
from Crypto.Cipher import AES

#秘钥,此处需要将字符串转为字节
key = b'0e9ec82fc54142de919f557a28bf637a'

#加密内容需要长达16位字符，所以进行空格拼接
def pad(text):
  while len(text) % 16 != 0:
    text += b' '
  return text

#加密秘钥需要长达16位字符，所以进行空格拼接
def pad_key(key):
  while len(key) % 16 != 0:
    key += b' '
  return key

#进行加密算法，模式ECB模式，把叠加完16位的秘钥传进来
aes = AES.new(pad_key(key), AES.MODE_ECB)
aa = {"action":"0","address":"","area":"","city":"","entContact":"daisy","entContactEmail":"ying.chen@tongfudun1111.com","entContactPhone":"18896551111","entId":810,"entName":"通付盾1111","entNo":"tongfudun1111","industry":16,"organizationId":"7d5d12318cf644bab92b16f0eebaf52c","orgnazationId":"7d5d12318cf644bab92b16f0eebaf52c","platform":"B","province":""}
#加密内容,此处需要将字符串转为字节
text=b'thisisjiamineirong'
# text = str(json.dumps(aa))
#进行内容拼接16位字符后传入加密类中，结果为字节类型
encrypted_text = aes.encrypt(pad(text))

print(encrypted_text)

#此处是为了验证是否能将字节转为字符串后，进行解密成功
#实际上a 就是 encrypted_text ，也就是加密后的内容
a = b'\x91\xbc\x7f*\xae\x02\xf8\x0eB\xc8\x98\x03z\xbb5@<(\xa1\x90\xad\x14veyn\x18\x18\xb5\xf7,\x9b'
# a = b'FB7DB864394160B719DA655CEB7400A7A5995A8DE187A2FA87BEB06CD3239CEEFBB19009818AF99DA71FB6A750537624665A8A0CA1AFE144A85F4C4475C413E81952D502722B7B6551F92A92C7C19644BC376D9D355C5F072C45F52F80FED8B5C25C828859BBB0F2F400249C851E96A2CD639F2A6E47305BB53B3EF1E77C80378D74E9E7FF663EFA610CCBB432D2163BC304EED03B56970F464B0FC8000EBA1DAF223D72D2A66A9994A370DBBC168A208666B287EAF06C4410D38C14FF78A1E802BFAB1AF7C06F5C849841827B78AF661C57154C758904B2305CA225F5DD6D952F90AB4059A4884C59BB5A5BFFD5722E52E4C76DA65BF1A7A95ECB99E1A03294223093E826F8B23C91342A1F1948CABE139503F4767B96ABAC2C89ED608F34749C107071B3B8588E38B43CA0036E879E382D490AB71F460575035FF965E7E69351A51F3A343A3C986AB6622112EE4FDE38CD0269AF5CEB7264222B365D81631D1FB96B52147F892B00CB3347AEAE2961'
#用aes对象进行解密，将字节类型转为str类型，错误编码忽略不计
de = str(aes.decrypt(a),encoding='utf-8',errors="ignore")

#获取str从0开始到文本内容的字符串长度。
print(de[:len(text)])


b'\x91\xbc\x7f*\xae\x02\xf8\x0eB\xc8\x98\x03z\xbb5@<(\xa1\x90\xad\x14veyn\x18\x18\xb5\xf7,\x9b'
thisisjiamineirong


In [40]:
import hashlib
from Crypto.Cipher import AES
from binascii import b2a_hex, a2b_hex
from pprint import pprint
import json

class Crypto():
    def __init__(self, key):
        tmp = hashlib.sha1(key).digest()
        tmp = hashlib.sha1(tmp).digest()[:16]
        self.key = tmp
        self.mode = AES.MODE_ECB

    # 加密函数，如果text不是16的倍数【加密文本text必须为16的倍数！】，那就补足为16的倍数
    def encrypt(self, text):
        cryptor = AES.new(self.key, self.mode)
        # 这里密钥key 长度必须为16（AES-128）、24（AES-192）、或32（AES-256）Bytes 长度.目前AES-128足够用
        length = 16
        count = len(text)

        if count % length != 0:
            add = length - (count % length)
        else:
            add = 0
        text = text + (b'\0' * add)
        self.ciphertext = cryptor.encrypt(text)
        # 因为AES加密时候得到的字符串不一定是ascii字符集的，输出到终端或者保存时候可能存在问题
        # 所以这里统一把加密后的字符串转化为16进制字符串
        return b2a_hex(self.ciphertext)

    # 解密后，去掉补足的空格用strip() 去掉
    def decrypt(self, text):
        cryptor = AES.new(self.key, self.mode)
        plain_text = cryptor.decrypt(a2b_hex(text))
        return plain_text.rstrip(b'\0')

    def decrypt_str(self, text):
        data = self.decrypt(text)
        print(data)
        text = data.decode().strip()

#         text = data.decode().strip().strip('\x06').strip('\x0c')
        return text
    
data = '86FE4705AD2A5B07654497F61FA183A2B1ADFF86DE9C88F9FC133D4A9509860FE2F216C3DFC437F5C9B9DB344F717794671183500DD5BD6615FD82355053AAE3849C034B93C3F34ECCFAFF7463CCEAA55D61CD42ED97C005FFFE56A0D975DE08423A813A28BB1A480D2EC39B2151FAB29778A447634D1F490CBE6AA02C6F80A5893950B860050835E3B91E32BC4CEE490E597AE3228775DC8BD8031A97D18AED7EC6192A5A4E1E84E55D4AC849C3174BB0BD842D98FE1541AAD4B50E3211B2E46F7D8979395ABD8DD967501B70CCC4171D126E67A5EDE01AD29CDC6173B15901511154EDC4FE97895405E327B7CB1ABC8E2EB5DE6BC89B2D52D726289BC38A8841D012F1130A701E43ABD6342D96B8E40FEED4671944DE31145028DCE28916A8D111592EA52A7512C7FC455E8E8A69849E118B7D75A6D529B22A9AC4F50EA4C65C00C245793989A3947839ED4AC4C9CF4ED2C99A2857D8BFA994C101C351A46F18458C2D03B77F959949E24D7B87620C0BF6184688795F5A16EE09F40F7DAD2D2E817665A9234468FF1BA9852846A93DC0B38EAA38CF906B33286E251D6E1F9E10756CF49872E4080E74DBBA8F3670CF9E4F32A94B4782868A2D80579BB56644'

cryptor = Crypto(b'0e9ec82fc54142de919f557a28bf637a')
info = cryptor.decrypt_str(data)
# pprint(info)

# pprint(json.loads(info))

b'{"action":0,"contactEmail":"ces@c.com","contactName":"\xe7\xae\xa1\xe7\x90\x86\xe5\x91\x98","contactPhone":"","entId":1,"entName":"\xe9\x80\x9a\xe4\xbb\x98\xe7\x9b\xbe\xe9\x9b\x86\xe5\x9b\xa2","entNo":"payegis","manager":"","managerEmail":"","managerPhone":"","organizationId":"1a84f6714f524c65b3e82fc4400276c4","orgnazationId":"1a84f6714f524c65b3e82fc4400276c4","platform":"B","secContactEmail":"","secContactName":"","secContactPhone":"","supporter":"","userName":"admin@payegis.com","user_code":"payegis:admin"}\x06\x06\x06\x06\x06\x06'


In [37]:
import memcache
from pprint import pprint


token = 'a7280010-6db4-48f7-b779-97271aaed871'

mc = memcache.Client(['192.168.109.108:11211'], debug=0)

res = mc.get(token)

pprint(res)

mc.set('abc', {'a':123, 'b':[1,2,4]})

print(mc.get('abc'))

ValueError: Unknown flags on get: 20

In [38]:
from pymemcache.client.base import Client  
  
client = Client(('192.168.109.108', 11211))  

result = client.get('a7280010-6db4-48f7-b779-97271aaed871')  

print(result)

b'{"account":"admin@payegis.com","sessionId":"6840E9F5BCF18B0A1391886AE133A68D","sessionIds":{"ba":"-1","clientMonitor":"-1","credit":"-1","deviceDID":"-1","fanqizha":"-1","fanqizha_t1":"-1","monitor":"-1","phoneBlackList":"-1","rra":"-1","secapp":"-1","slide":"-1","sms":"-1","websec":"-1"}}'
